In [3]:
# Import packages
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import re
from shapely.geometry import Polygon, MultiPolygon
from rasterstats import zonal_stats
from osgeo import gdal
import rasterio
import rasterstats
import time

In [4]:
yields = gpd.read_file(r'C:\Users\mieke\Documents\Msc Thesis\Notebooks Python\Final\yields_geometry.geojson') # All yield observations for paddy, maize and soghrum
obs_calculation = gpd.read_file(r'C:\Users\mieke\Documents\Msc Thesis\Notebooks Python\Final\obs_calculation.geojson') # All unique geometries for each crop type per year, season, unit
# TIME: 4.5min

In [5]:
yields_copy = yields.copy()
obs_calculation_copy = obs_calculation.copy()
# yields = yields_copy.copy()
# obs_calculation = obs_calculation_copy.copy()

In [6]:
obs_paddy = obs_calculation[obs_calculation['Crop'] == 'paddy'] # 7285
obs_maize = obs_calculation[obs_calculation['Crop'] == 'maize (makka)'] # 6008
obs_sorghum = obs_calculation[obs_calculation['Crop'] == 'sorghum (jowar/great millet)'] # 4788

yields_paddy = yields[yields['Crop'] == 'paddy'] # 8461
yields_maize = yields[yields['Crop'] == 'maize (makka)'] # 7480 
yields_sorghum = yields[yields['Crop'] == 'sorghum (jowar/great millet)'] # 5411

In [7]:
# Import all dataframes including monthly/daily values per hobli/gram panchayat
maize_rabi_false = gpd.read_file(r'D:\other_thesis\maize_rabi_false.geojson') # Save the derived datframe as it takes a long time to run
maize_kharif_false = gpd.read_file(r'D:\other_thesis\maize_kharif_false.geojson') # Save the derived datframe as it takes a long time to run

# # True wrt LST and LSTN
# maize_rabi_true = gpd.read_file(r'D:\other_thesis\maize_rabi_true.geojson') # Save the derived datframe as it takes a long time to run
# maize_kharif_true = gpd.read_file(r'D:\other_thesis\maize_kharif_true.geojson') # Save the derived datframe as it takes a long time to run

maize_rabi_precipitation = gpd.read_file(r'D:\other_thesis\maize_rabi_precipitation.geojson') # Save the derived datframe as it takes a long time to run
maize_kharif_precipitation = gpd.read_file(r'D:\other_thesis\maize_kharif_precipitation.geojson') # Save the derived datframe as it takes a long time to run

maize_rabi_ssm = gpd.read_file(r'D:\other_thesis\maize_rabi_ssm.geojson') # Save the derived datframe as it takes a long time to run
maize_kharif_ssm = gpd.read_file(r'D:\other_thesis\maize_kharif_ssm.geojson') # Save the derived datframe as it takes a long time to run

maize_rabi_susm = gpd.read_file(r'D:\other_thesis\maize_rabi_susm.geojson') # Save the derived datframe as it takes a long time to run
maize_kharif_susm = gpd.read_file(r'D:\other_thesis\maize_kharif_susm.geojson') # Save the derived datframe as it takes a long time to run

maize_rabi_soiltype = gpd.read_file(r'D:\other_thesis\maize_rabi_soiltype.geojson')
maize_kharif_soiltype = gpd.read_file(r'D:\other_thesis\maize_kharif_soiltype.geojson')
# TIME: 5.5min

In [8]:
print(maize_rabi_false.isna().sum().sum()) # 624
#print(maize_rabi_true.isna().sum().sum()) # 603
# 21
print(maize_kharif_false.isna().sum().sum()) # 20326
#print(maize_kharif_true.isna().sum().sum()) # 19639
# 687
print(np.size(maize_rabi_false.iloc[:,10:-1])) # 71700
print(np.size(maize_kharif_false.iloc[:,10:-1])) # 367080

624
603
20326
19639
71700
367080


In [12]:
maize_rabi_precipitation.iloc[:,10:].isna().sum().sum() # 476
maize_rabi_precipitation.iloc[:,10:] = maize_rabi_precipitation.iloc[:,10:].fillna(0)
maize_rabi_precipitation.iloc[:,10:].isna().sum().sum() # 0 

maize_kharif_precipitation.iloc[:,10:].isna().sum().sum() # 25466
maize_kharif_precipitation.iloc[:,10:] = maize_kharif_precipitation.iloc[:,10:].fillna(0)
maize_kharif_precipitation.iloc[:,10:].isna().sum().sum() # 0 

0

In [243]:
maize_kharif_false.iloc[[16,17,19,29,90],9:17].sort_values('Average Yield(Kg/Ha)').head(30)

,Average Yield(Kg/Ha),NDVI_05_median,NDVI_06_median,NDVI_07_median,NDVI_08_median,NDVI_09_median,NDVI_10_median,NDVI_11_median
29,646.45,0.383750,0.370425,0.350575,0.429675,0.575275,0.49345,0.43375
17,860.60,0.426025,0.295825,0.260900,0.505125,0.546000,0.57650,0.53710
19,5261.52,0.395875,0.335400,0.258625,0.540425,0.520825,0.68025,0.57055
16,6095.71,0.434950,0.404700,0.481850,0.438050,0.545450,0.59730,0.54570


In [29]:
# We create a dataframe containing all parameter values for each observations in the rabi season
# For rabi, we use october - may
df_param_rabi = maize_rabi_false.iloc[:,[0,1,2,3,4,5,6,7,8,-1]]

df_param_rabi['NDVI_avg'] = maize_rabi_false.iloc[:,11:19].mean(axis=1) 
df_param_rabi['NDVI_max'] = maize_rabi_false.iloc[:,11:19].max(axis=1)

df_param_rabi['EVI_avg'] = maize_rabi_false.iloc[:,21:29].mean(axis=1)
df_param_rabi['EVI_max'] = maize_rabi_false.iloc[:,21:29].max(axis=1)

df_param_rabi['FAPAR_avg'] = maize_rabi_false.iloc[:,31:39].mean(axis=1)
df_param_rabi['FAPAR_max'] = maize_rabi_false.iloc[:,31:39].max(axis=1)

df_param_rabi['LAI_avg'] = maize_rabi_false.iloc[:,41:49].mean(axis=1)
df_param_rabi['LAI_max'] = maize_rabi_false.iloc[:,41:49].max(axis=1)

df_param_rabi['SIF_avg'] = maize_rabi_false.iloc[:,51:59].mean(axis=1)
df_param_rabi['SIF_max'] = maize_rabi_false.iloc[:,51:59].max(axis=1)

df_param_rabi['SSM_med'] = maize_rabi_false.iloc[:,61:69].median(axis=1)
df_param_rabi['SSM_max'] = maize_rabi_false.iloc[:,61:69].max(axis=1)

df_param_rabi['SUSM_med'] = maize_rabi_false.iloc[:,71:79].median(axis=1)
df_param_rabi['SUSM_max'] = maize_rabi_false.iloc[:,71:79].max(axis=1)

df_param_rabi['LST_med'] = maize_rabi_false.iloc[:,81:89].median(axis=1)
df_param_rabi['LST_max'] = maize_rabi_false.iloc[:,81:89].max(axis=1)

df_param_rabi['LSTN_med'] = maize_rabi_false.iloc[:,91:99].median(axis=1)
df_param_rabi['LSTN_max'] = maize_rabi_false.iloc[:,91:99].max(axis=1)

df_param_rabi['Prec_acc'] = maize_rabi_precipitation.iloc[:,40:-31].sum(axis=1)
df_param_rabi['Prec_non'] = maize_rabi_precipitation.iloc[:,40:-31].isin([0]).sum(axis=1)

df_param_rabi['Soiltype'] = maize_rabi_soiltype['Soiltype_majority']

df_param_rabi.head(1)

c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

,Year,Season,Insurance Unit,shapeName,Gram Panchayat/Hobli,District,Taluk,Crop,IRR_RF,geometry,...,SSM_max,SUSM_med,SUSM_max,LST_med,LST_max,LSTN_med,LSTN_max,Prec_acc,Prec_non,Soiltype
0,2016,Rabi,adahalli,adahalli,gp,belagavi,athani,maize (makka),irr,"POLYGON ((75.19845 16.84287, 75.19846 16.84273...",...,11.16288,25.384209,99.093796,38.423315,49.389217,21.10161,26.759129,134.098302,226,37.0


In [42]:
# We create a dataframe containing all parameter values for each observations in the rabi season
# For kharif, we use may - november
df_param_kharif = maize_kharif_false.iloc[:,[0,1,2,3,4,5,6,7,8,-1]]

df_param_kharif['NDVI_avg'] = maize_kharif_false.iloc[:,10:17].mean(axis=1)
df_param_kharif['NDVI_max'] = maize_kharif_false.iloc[:,10:17].max(axis=1)

df_param_kharif['EVI_avg'] = maize_kharif_false.iloc[:,17:24].mean(axis=1)
df_param_kharif['EVI_max'] = maize_kharif_false.iloc[:,17:24].max(axis=1)

df_param_kharif['FAPAR_avg'] = maize_kharif_false.iloc[:,24:31].mean(axis=1)
df_param_kharif['FAPAR_max'] = maize_kharif_false.iloc[:,24:31].max(axis=1)

df_param_kharif['LAI_avg'] = maize_kharif_false.iloc[:,31:38].mean(axis=1)
df_param_kharif['LAI_max'] = maize_kharif_false.iloc[:,31:38].max(axis=1)

df_param_kharif['SIF_avg'] = maize_kharif_false.iloc[:,38:45].mean(axis=1)
df_param_kharif['SIF_max'] = maize_kharif_false.iloc[:,38:45].max(axis=1)

df_param_kharif['SSM_med'] = maize_kharif_false.iloc[:,45:52].median(axis=1)
df_param_kharif['SSM_max'] = maize_kharif_false.iloc[:,45:52].max(axis=1)

df_param_kharif['SUSM_med'] = maize_kharif_false.iloc[:,52:59].median(axis=1)
df_param_kharif['SUSM_max'] = maize_kharif_false.iloc[:,52:59].max(axis=1)

df_param_kharif['LST_med'] = maize_kharif_false.iloc[:,59:66].median(axis=1)
df_param_kharif['LST_max'] = maize_kharif_false.iloc[:,59:66].max(axis=1)

df_param_kharif['LSTN_med'] = maize_kharif_false.iloc[:,66:73].median(axis=1)
df_param_kharif['LSTN_max'] = maize_kharif_false.iloc[:,66:73].max(axis=1)

df_param_kharif['Prec_acc'] = maize_kharif_precipitation.iloc[:,10:-1].sum(axis=1)
df_param_kharif['Prec_non'] = maize_kharif_precipitation.iloc[:,10:-1].isin([0]).sum(axis=1)

df_param_kharif['Soiltype'] = maize_kharif_soiltype['Soiltype_majority']

df_param_kharif.head(1)

c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

,Year,Season,Insurance Unit,shapeName,Gram Panchayat/Hobli,District,Taluk,Crop,IRR_RF,geometry,...,SSM_max,SUSM_med,SUSM_max,LST_med,LST_max,LSTN_med,LSTN_max,Prec_acc,Prec_non,Soiltype
0,2016,Kharif,a.hosahalli,a.hosahalli,gp,hassan,hassan,maize (makka),irr,"POLYGON ((76.29514 13.07897, 76.29570 13.07768...",...,17.593796,76.167831,115.859352,31.26072,39.744843,18.608072,22.081055,458.101643,154,33.0


In [178]:
# Combine rabi and kharif dataframes
# Besides, we create dummies for the irrigation method, the season, and the soil type.
df_param = pd.concat([df_param_rabi, df_param_kharif], ignore_index=True)
irr_dummies = pd.get_dummies(df_param['IRR_RF'])
irr_dummies = irr_dummies.rename(columns = {'irr': 'Irrigated'})
season_dummies = pd.get_dummies(df_param['Season'])
soiltype_dummies = pd.get_dummies(df_param['Soiltype'])#,columns = ['I', 'Vp', 'WR', 'Bv', 'Ne', 'Vc', 'Ap', 'Lc'])
soiltype_dummies = soiltype_dummies.rename(columns = {3.0: 'Soil_I', 4.0: 'Soil_Vp', 17.0: 'Soil_WR', 27.0: 'Soil_Bv', 33.0: 'Soil_Ne', 37.0: 'Soil_Vc', 39.0: 'Soil_Ap', 43.0: 'Soil_Lc'})
# The 8 types of soil in Karnataka:
# I: Lithosols
# Vp: Pellic Vertisols
# WR: Water Bodies
# Bv: Vertic Cambisols
# Ne: Eutric Nitosols
# Vc: Chromic Vertisols
# Ap: Plinthis Acrisols
# Lc: Chromic Luvisols

In [179]:
# Add dummy variables to the model
df_param = pd.concat([df_param, irr_dummies, season_dummies, soiltype_dummies], axis = 1)

In [180]:
# Merge the feature dataframe with the yield observations
rf_params = yields_maize.merge(df_param, how = 'left', on = ['Year', 'Season', 'Insurance Unit', 'shapeName', 'Gram Panchayat/Hobli', 'District', 'Taluk', 'Crop', 'geometry'])
rf_params.head(1)

,Year,Season,Insurance Unit,shapeName,Gram Panchayat/Hobli,District,Taluk,Crop,IRR_RF_x,Average Yield(Kg/Ha),...,Kharif,Rabi,Soil_I,Soil_Vp,Soil_WR,Soil_Bv,Soil_Ne,Soil_Vc,Soil_Ap,Soil_Lc
0,2016,Kharif,a.hosahalli,a.hosahalli,gp,hassan,hassan,maize (makka),irr,2292.11,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [182]:
# Note that as we only have 51 observations during Zaid and the timespan is not clear, we drop them from the dataframe
rf_params = rf_params[rf_params['Season'] != 'Summer'] # 51 dropped
rf_params = rf_params.sort_values(['Year', 'Season'], ascending = [True, False], ignore_index=True) # Sort the dataframe based on Year and Season (needed for cross validation)

In [ ]:
rf_params.columns
rf_params = rf_params.drop(['Season', 'Rabi', 'IRR_RF_y', 'IRR_RF_x', 'rf', 'Soiltype'], axis = 1)


In [185]:
null_data = rf_params[rf_params.isnull().any(axis=1)]
null_data_df = pd.DataFrame(null_data.drop(['geometry'], axis = 1)) # 238 for units, 1115 for yields
rf_params = rf_params.dropna(axis=0) # drop observations with missing data (1115) 

In [145]:
df_param.isna().sum().sum() # 1138
len(df_param_kharif['Soiltype'].unique()) # 9 (3 missing values)
len(df_param_rabi['Soiltype'].unique()) # 6 (0 missing values)
df_param = df_param.dropna(axis=0)  # drop all rows including NaNs, # 5723 obs left

In [148]:
# Save the final dataframe
# df_param.to_file(r'D:\other_thesis\df_param.geojson', driver="GeoJSON")
rf_params.to_file(r'D:\other_thesis\rf_params.geojson', driver="GeoJSON")

c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [10]:
# FOR LATER USE
# df = pd.DataFrame(np.array([[1, 1], [2, 10], [3, 100], [4, 100]]), columns=['a', 'b'])
# print(np.transpose(df).quantile(0.9, interpolation='nearest'))
# print(df.quantile(0.9, interpolation='nearest', axis=1))
# print(np.transpose(df).quantile(0.5))#, interpolation='nearest'))
# print(np.transpose(df)[np.transpose(df) > 100].count())
# print(np.transpose(df).mean(axis=0))
# df

# High NDVI values might not correspond to maize, so maybe some other crop is grown there, less of maize
# (np.nanmax(maize_rabi_false[['NDVI_09_median', 'NDVI_10_median']].values, axis=1) - maize_rabi_false.iloc[:,10]#).head(20) #11 - 10 positief verschil betekent dus een increase in NDVI. Dus hoe hoger hoe positiever.